In [ ]:
# CELLULE 1 - VÉRIFICATION ET IMPORTATION
print("🚗 PROJET DE PRÉDICTION DE PRIX DE VOITURES - DÉMARRAGE")
print("=" * 55)

# Importation des bibliothèches
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, r2_score
import warnings
warnings.filterwarnings('ignore')

# Configuration de l'affichage
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)

print("✅ Bibliothèches importées avec succès!")
print(f"📦 Pandas version: {pd.__version__}")
print(f"📦 NumPy version: {np.__version__}")

In [ ]:
# CELLULE 2 - CRÉATION DU DATASET DE VOITURES
print("📊 CRÉATION DU DATASET DE VOITURES")
print("=" * 40)

np.random.seed(42)  # Pour la reproductibilité
n_samples = 1000

# Création de données réalistes
car_data = {
    'Marque': np.random.choice(['Toyota', 'Honda', 'Ford', 'BMW', 'Mercedes', 'Audi', 'Hyundai', 'Nissan'], 
                              n_samples, p=[0.15, 0.15, 0.15, 0.1, 0.1, 0.1, 0.15, 0.1]),
    'Année': np.random.randint(2005, 2023, n_samples),
    'Kilométrage': np.random.randint(5000, 150000, n_samples),
    'Puissance': np.random.randint(80, 400, n_samples),
    'Carburant': np.random.choice(['Essence', 'Diesel', 'Hybride', 'Électrique'], 
                                 n_samples, p=[0.5, 0.3, 0.15, 0.05]),
    'Transmission': np.random.choice(['Manuelle', 'Automatique'], n_samples, p=[0.4, 0.6]),
    'Type': np.random.choice(['Berline', 'SUV', 'Compacte', 'Break', 'Coupé'], n_samples),
    'Prix': np.random.randint(5000, 60000, n_samples)  # Variable cible
}

df = pd.DataFrame(car_data)

# Affichage des informations
print(f"✅ Dataset créé: {df.shape[0]} voitures, {df.shape[1]} caractéristiques")
print("\n📋 Aperçu des 5 premières voitures:")
display(df.head())

print("\n📈 Statistiques descriptives:")
display(df.describe())

print("\n🔍 Informations sur le dataset:")
df.info()

In [ ]:
# CELLULE 3 - ANALYSE EXPLORATOIRE DES DONNÉES
print("🔍 ANALYSE EXPLORATOIRE DES DONNÉES (EDA)")
print("=" * 50)

# Configuration des graphiques
fig, axes = plt.subplots(2, 3, figsize=(20, 12))
fig.suptitle('ANALYSE DES PRIX DE VOITURES - TABLEAU DE BORD', fontsize=16, fontweight='bold')

# 1. Distribution des prix
axes[0,0].hist(df['Prix'], bins=30, edgecolor='black', alpha=0.7, color='skyblue')
axes[0,0].set_title('Distribution des Prix des Voitures')
axes[0,0].set_xlabel('Prix (€)')
axes[0,0].set_ylabel('Nombre de Voitures')
axes[0,0].grid(True, alpha=0.3)

# 2. Prix moyen par marque
marque_prix = df.groupby('Marque')['Prix'].mean().sort_values(ascending=False)
axes[0,1].bar(marque_prix.index, marque_prix.values, color='lightcoral')
axes[0,1].set_title('Prix Moyen par Marque')
axes[0,1].set_xlabel('Marque')
axes[0,1].set_ylabel('Prix Moyen (€)')
axes[0,1].tick_params(axis='x', rotation=45)

# 3. Évolution des prix par année
annee_prix = df.groupby('Année')['Prix'].mean()
axes[0,2].plot(annee_prix.index, annee_prix.values, marker='o', linewidth=2, color='green')
axes[0,2].set_title('Évolution des Prix par Année')
axes[0,2].set_xlabel('Année')
axes[0,2].set_ylabel('Prix Moyen (€)')
axes[0,2].grid(True, alpha=0.3)

# 4. Prix vs Kilométrage
axes[1,0].scatter(df['Kilométrage'], df['Prix'], alpha=0.6, color='orange')
axes[1,0].set_title('Relation Prix vs Kilométrage')
axes[1,0].set_xlabel('Kilométrage (km)')
axes[1,0].set_ylabel('Prix (€)')
axes[1,0].grid(True, alpha=0.3)

# 5. Prix vs Puissance
axes[1,1].scatter(df['Puissance'], df['Prix'], alpha=0.6, color='purple')
axes[1,1].set_title('Relation Prix vs Puissance')
axes[1,1].set_xlabel('Puissance (ch)')
axes[1,1].set_ylabel('Prix (€)')
axes[1,1].grid(True, alpha=0.3)

# 6. Prix par type de carburant
carburant_prix = df.groupby('Carburant')['Prix'].mean().sort_values(ascending=False)
axes[1,2].bar(carburant_prix.index, carburant_prix.values, color='gold')
axes[1,2].set_title('Prix Moyen par Type de Carburant')
axes[1,2].set_xlabel('Type de Carburant')
axes[1,2].set_ylabel('Prix Moyen (€)')

plt.tight_layout()
plt.show()

# Matrice de corrélation (pour les variables numériques)
print("\n📊 MATRICE DE CORRÉLATION:")
numerical_df = df.select_dtypes(include=[np.number])
plt.figure(figsize=(10, 8))
sns.heatmap(numerical_df.corr(), annot=True, cmap='coolwarm', center=0, square=True)
plt.title('Matrice de Corrélation des Variables Numériques')
plt.tight_layout()
plt.show()

print("✅ Analyse exploratoire terminée!")

In [ ]:
# CELLULE 4 - PRÉTRAITEMENT DES DONNÉES
print("⚙️ PRÉTRAITEMENT DES DONNÉES")
print("=" * 35)

# Copie du dataset pour le prétraitement
df_processed = df.copy()

# Feature Engineering
df_processed['Âge'] = 2023 - df_processed['Année']
df_processed['Km_Par_An'] = df_processed['Kilométrage'] / np.maximum(df_processed['Âge'], 1)

# Encodage des variables catégorielles
df_processed = pd.get_dummies(df_processed, columns=['Marque', 'Carburant', 'Transmission', 'Type'])

# Séparation des features et de la target
X = df_processed.drop('Prix', axis=1)
y = df_processed['Prix']

# Division en train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("✅ Prétraitement terminé!")
print(f"📊 Jeu d'entraînement: {X_train.shape[0]} échantillons")
print(f"📊 Jeu de test: {X_test.shape[0]} échantillons")
print(f"🔢 Nombre de features: {X_train.shape[1]}")

In [ ]:
# CELLULE 5 - ENTRAÎNEMENT DES MODÈLES
print("🤖 ENTRAÎNEMENT DES MODÈLES DE MACHINE LEARNING")
print("=" * 55)

# Initialisation des modèles
models = {
    'Régression Linéaire': LinearRegression(),
    'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42, max_depth=10)
}

results = {}

print("🔧 Entraînement en cours...\n")

for name, model in models.items():
    print(f"🧠 Entraînement du modèle: {name}")
    
    # Entraînement
    model.fit(X_train, y_train)
    
    # Prédictions
    y_pred = model.predict(X_test)
    
    # Métriques d'évaluation
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    # Stockage des résultats
    results[name] = {
        'model': model,
        'predictions': y_pred,
        'mae': mae,
        'r2': r2
    }
    
    print(f"✅ {name} - Entraînement terminé!")
    print(f"   📏 Erreur Absolue Moyenne (MAE): {mae:,.2f} €")
    print(f"   📊 Score R²: {r2:.4f}")
    print("-" * 50)

print("🎯 Tous les modèles sont entraînés!")

In [ ]:
# CELLULE 6 - ÉVALUATION ET COMPARAISON DES MODÈLES
print("📈 COMPARAISON DES PERFORMANCES DES MODÈLES")
print("=" * 50)

# Tableau de comparaison
comparison_df = pd.DataFrame({
    'Modèle': list(results.keys()),
    'MAE (€)': [results[model]['mae'] for model in results],
    'Score R²': [results[model]['r2'] for model in results]
})

print("📋 TABLEAU COMPARATIF:")
display(comparison_df)

# Sélection du meilleur modèle
best_model_name = max(results.keys(), key=lambda x: results[x]['r2'])
best_model = results[best_model_name]['model']
best_r2 = results[best_model_name]['r2']

print(f"\n🏆 MEILLEUR MODÈLE: {best_model_name}")
print(f"🎯 Score R²: {best_r2:.4f}")
print(f"📏 Erreur moyenne: ±{results[best_model_name]['mae']:,.2f} €")

# Importance des features (pour Random Forest)
if best_model_name == 'Random Forest':
    print("\n🔍 IMPORTANCE DES CARACTÉRISTIQUES:")
    feature_importance = pd.DataFrame({
        'Caractéristique': X.columns,
        'Importance': best_model.feature_importances_
    }).sort_values('Importance', ascending=False).head(10)
    
    display(feature_importance)
    
    # Visualisation de l'importance des features
    plt.figure(figsize=(12, 6))
    sns.barplot(data=feature_importance, x='Importance', y='Caractéristique')
    plt.title('Top 10 des Caractéristiques les Plus Importantes')
    plt.xlabel('Importance')
    plt.tight_layout()
    plt.show()

In [ ]:
# CELLULE 7 - VISUALISATION DES RÉSULTATS
print("📊 VISUALISATION DES PRÉDICTIONS")
print("=" * 40)

# Configuration des graphiques
fig, axes = plt.subplots(1, 2, figsize=(18, 6))

# Graphique Random Forest
y_pred_rf = results['Random Forest']['predictions']
axes[0].scatter(y_test, y_pred_rf, alpha=0.6, color='blue', label='Prédictions')
axes[0].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2, label='Prédiction Parfaite')
axes[0].set_xlabel('Prix Réel (€)')
axes[0].set_ylabel('Prix Prédit (€)')
axes[0].set_title('Random Forest: Prix Réel vs Prix Prédit')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Graphique Régression Linéaire
y_pred_lr = results['Régression Linéaire']['predictions']
axes[1].scatter(y_test, y_pred_lr, alpha=0.6, color='green', label='Prédictions')
axes[1].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2, label='Prédiction Parfaite')
axes[1].set_xlabel('Prix Réel (€)')
axes[1].set_ylabel('Prix Prédit (€)')
axes[1].set_title('Régression Linéaire: Prix Réel vs Prix Prédit')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Analyse des résidus
print("\n🔎 ANALYSE DES RÉSIDUS:")
residuals = y_test - y_pred_rf

plt.figure(figsize=(15, 4))

plt.subplot(1, 3, 1)
plt.scatter(y_pred_rf, residuals, alpha=0.6)
plt.axhline(y=0, color='red', linestyle='--', linewidth=2)
plt.xlabel('Prix Prédit (€)')
plt.ylabel('Résidus (€)')
plt.title('Graphique des Résidus')
plt.grid(True, alpha=0.3)

plt.subplot(1, 3, 2)
plt.hist(residuals, bins=30, edgecolor='black', alpha=0.7)
plt.xlabel('Résidus (€)')
plt.ylabel('Fréquence')
plt.title('Distribution des Résidus')
plt.grid(True, alpha=0.3)

plt.subplot(1, 3, 3)
plt.boxplot(residuals)
plt.ylabel('Résidus (€)')
plt.title('Boîte à Moustaches des Résidus')

plt.tight_layout()
plt.show()

In [ ]:
# CELLULE 8 - PRÉDICTIONS SUR DE NOUVELLES VOITURES
print("🎯 PRÉDICTIONS SUR DE NOUVELLES VOITURES")
print("=" * 45)

def predire_prix(marque, annee, kilometrage, puissance, carburant, transmission, type_voiture):
    """Fonction pour prédire le prix d'une voiture"""
    
    # Création du dictionnaire d'entrée
    input_data = {
        'Année': annee,
        'Kilométrage': kilometrage,
        'Puissance': puissance,
        'Âge': 2023 - annee,
        'Km_Par_An': kilometrage / max(2023 - annee, 1)
    }
    
    # Ajout de toutes les colonnes dummy avec des 0
    for col in X.columns:
        if col not in input_data:
            # Vérifier si cette colonne correspond à nos inputs
            if marque in col or carburant in col or transmission in col or type_voiture in col:
                input_data[col] = 1
            else:
                input_data[col] = 0
    
    # Création du DataFrame
    input_df = pd.DataFrame([input_data])
    input_df = input_df[X.columns]  # Assurer le bon ordre des colonnes
    
    # Prédiction
    prix_pred = best_model.predict(input_df)[0]
    
    return prix_pred

# Exemples de prédictions
print("🚗 EXEMPLES DE PRÉDICTIONS:\n")

exemples = [
    ('Toyota', 2020, 25000, 150, 'Essence', 'Automatique', 'Berline'),
    ('BMW', 2019, 40000, 300, 'Essence', 'Automatique', 'SUV'),
    ('Ford', 2015, 80000, 120, 'Diesel', 'Manuelle', 'Compacte'),
    ('Mercedes', 2021, 15000, 250, 'Hybride', 'Automatique', 'Berline'),
    ('Audi', 2018, 60000, 200, 'Diesel', 'Automatique', 'SUV')
]

for marque, annee, km, puissance, carburant, transmission, type_voiture in exemples:
    prix = predire_prix(marque, annee, km, puissance, carburant, transmission, type_voiture)
    print(f"• {annee} {marque} {type_voiture} ({puissance}ch, {km:,} km): {prix:,.0f} €")

print(f"\n🎉 MODÈLE FINAL: {best_model_name}")
print(f"📊 PRÉCISION: R² = {best_r2:.4f}")
print(f"🎯 ERREUR MOYENNE: ±{results[best_model_name]['mae']:,.0f} €")
print("\n✅ PROJET TERMINÉ AVEC SUCCÈS! 🚗💨")

In [ ]:
# TEST D'IMPORTATION
print("=== TEST DE DÉMARRAGE ===")

import pandas as pd
import numpy as np

print("✅ Pandas et NumPy importés!")
print(f"Version Pandas: {pd.__version__}")

# Création simple
data = {'Prix': [10000, 20000, 30000], 'Année': [2015, 2018, 2020]}
df = pd.DataFrame(data)

print("\n📊 DataFrame créé:")
print(df)

print("\n=== TEST RÉUSSI ===")